## Doc 360 Managed Care

In [1]:
import polars as pl
import gc
import pandas as pd
from datetime import datetime, timedelta,date
import json
import numpy as np

In [2]:
# load variables from JSON
with open('vars_wk.json', 'r') as json_file:
    js = json.load(json_file)

bucket = js['bucket']
data_date = js['data_date']
monthly_data_date = js['monthly_data_date']

dflib = f's3://{bucket}/BIT/dataframes/'
pln = f's3://{bucket}/PYADM/weekly/archive/{data_date}/plantrak/' #20240705  {data_date}
mpln = f's3://{bucket}/PYADM/monthly/archive/{monthly_data_date}/plantrak/'

In [3]:
# Utility Functions -
def load(df, lib=dflib):
    globals()[df] = pl.read_parquet(f'{lib}{df}.parquet')

In [4]:
# Imporing Dependencies
prod_mapping = pl.read_csv(f's3://{bucket}/BIT/docs/productmapping_pybit.txt',separator='|')
geo_code_mapper = pl.from_pandas(pd.read_excel(f's3://{bucket}/BIT/docs/geo_id_full.xlsx'))
load('mp_spec_seg_dec')
load('MASTER_UNI')
fetch_products = ['LI1','LI2','LI3','TRU','AMT','LAC','MOT','LUB','IRL']

---

#### Importing Plantrak - Getting 6 Month Volume | Current and Prior

In [5]:
#PLANTRAK - 6 MONTH TUF at PROD , IID , PLAN LEVEL
ln = (
    pl.read_parquet(mpln+'LAX_N.parquet',columns=['IID','MonthKey','PFAM_CD','PROD_CD','PlanID','TUF']) #read req cols only
    .rename({'MonthKey':'PeriodKey'})
    .filter(pl.col('PROD_CD').is_in(fetch_products)) #only keep data for BIT products
    .with_columns(pl.col('PeriodKey').cast(pl.Utf8).str.to_date("%Y%m%d")) #Convert Categorical column Back to date
)

cut_off_date = ln['PeriodKey'].unique().sort(descending=True)[5] #filter date to only keep 6 months of data

ln_cur6m = (
    ln
    .filter(pl.col('PeriodKey')>= cut_off_date)
    .group_by(['IID','PlanID','PFAM_CD','PROD_CD']).agg(TUF_c = pl.col('TUF').sum()).unique()
)

ln_pre6m = (
    ln
    .filter(
        (pl.col('PeriodKey') <= (ln['PeriodKey'].unique().sort(descending=True)[6])) &(pl.col('PeriodKey') >= (ln['PeriodKey'].unique().sort(descending=True)[11]))
    )
    .group_by(['IID','PlanID','PFAM_CD','PROD_CD']).agg(TUF_p = pl.col('TUF').sum()).unique()
)

ln = ln_cur6m.join(
    ln_pre6m,
    on = ['IID','PlanID','PFAM_CD','PROD_CD'],how = 'left'
)

#### Importing Formulary -
- also splitting formuary into 2 sub datasets -
- fm_3 : unique on IWRD_FGN_NAME , PFAM_CD
- fm_1 : Unique on PlanID

In [6]:
#FORMULARY
group_type_mapping = {
    'HIX' : 'Commerical','Com' : 'Commerical','Cash' : 'Cash','Voucher':'Voucher',
    'FFS' : 'FFS','Mgd Medicaid' : 'Mgd Medicaid','Part D' : 'Part D','MAC A' : 'Others',
}

def classify_plan_class(status):
    status = status.upper()
    if status[:7] == "COVERED" or status[:6] == "ON PDL":
        return "COVERED"
    elif status[:9] == "PREFERRED":
        return "PREFERRED"
    elif status[:13] == "NON-PREFERRED":
        return "NON PREFERRED"
    elif status[:7] == "NON-PDL" or status[:11] == "NOT COVERED":
        return "NOT COVERED"
    else:
        return "N_A"

fm = pl.read_parquet(pln+'FORMULARY.parquet',columns = ['IMS_PLAN_ID','GROUP_TYPE','FORMULARY_GROUP_STATUS','PFAM_CD','PFAM_NAME','IRWD_FGN_NAME','BRAND'])
fm = fm.with_columns(
        pl.when(pl.col('BRAND')=='IBR')
        .then(pl.lit('IRL'))
        .otherwise(pl.col('BRAND'))
        .alias('BRAND'))
    
fm = fm.filter((pl.col('PFAM_CD')==(pl.col('BRAND'))) | (pl.col('BRAND')==''))
fm = (
    fm
    .with_columns(
        pl.col('GROUP_TYPE').map_elements(lambda x: group_type_mapping.get(x,'Others'), return_dtype=pl.Utf8) #NOTE : IF new plan types flow , they will go to Others by default
        .fill_null('Others')
        .alias('plan_type'),
        pl.col('IMS_PLAN_ID').cast(pl.Int64)
    )
    .rename({'IMS_PLAN_ID':'PlanID'})
    .drop('GROUP_TYPE')
    .with_columns(pl.col('FORMULARY_GROUP_STATUS').fill_null(pl.lit('N_A')))
    .with_columns(pl.col('FORMULARY_GROUP_STATUS').map_elements(classify_plan_class,return_dtype=pl.String).alias('plan_class'))
    .drop('FORMULARY_GROUP_STATUS')
    # .join(prod_mapping[['product_id','code']],left_on = 'PFAM_CD',right_on='code',how='left').drop('PFAM_CD')#change 
    # .filter(pl.col('product_id').is_not_null())# change
    .unique()
)
fm = fm.with_columns(
    pl.when(pl.col("PlanID") == 13670614)
    .then(pl.lit('Others'))
    .otherwise(pl.col("plan_type"))
    .alias("plan_type")
)
fm_1 = fm.select(['PlanID','IRWD_FGN_NAME','plan_type']).unique(subset='PlanID')
# fm_2 = fm.select(['PlanID','PFAM_CD','PFAM_NAME','BRAND','plan_type','plan_class'])#change

fm_3 = (
    fm
    .select('PFAM_CD','IRWD_FGN_NAME','plan_class').unique()
    .group_by(['IRWD_FGN_NAME','PFAM_CD'])
    .agg(
        pl.col('plan_class').unique().str.concat(' / ').alias('plan_class')
    )
    .with_columns(pl.col('plan_class').str.to_titlecase())
)

#### Processing

In [7]:
#### Combining Plantrak with fm_1
#### Followed by join with fm_3
# Join Steps -
ln_1 = (
    ln.join(fm_1,on='PlanID',how='left')
     .with_columns(
        pl.col('plan_type').fill_null(pl.lit('Others')),
    )
)

ln_2 = (
    ln_1.join(fm_3, on=['IRWD_FGN_NAME', 'PFAM_CD'], how='left')
    .with_columns(
        pl.col('plan_class').fill_null(pl.lit('N_a'))
    )
)

# Any PlanIds startign with -0000002 should be excluded
ln_2 = ln_2.with_columns(ln_2["PlanID"].cast(pl.Utf8).alias("PlanID")) \
                         .filter(~pl.col("PlanID").str.slice(0, 6).eq("000002")) \
                         .with_columns(pl.col("PlanID").cast(pl.Int64).alias("PlanID"))

# Removing Any Voucher Records -
ln_2 = ln_2.filter(~pl.col('plan_type').is_in(['Voucher'])).join(prod_mapping[['product_id','code']],left_on = 'PROD_CD',right_on='code',how='left').drop('PROD_CD')

#### Removing Medicaid and FFS -
ln1_reduced = ln_2.filter(~pl.col('plan_type').is_in(['Mgd Medicaid', 'FFS']))

In [8]:
# Adding Parent Product Rows -
ln3 = ln1_reduced.select(['IID','PlanID','product_id','IRWD_FGN_NAME','plan_type','plan_class','TUF_c','TUF_p'])
ln3 = ln3.join(prod_mapping[['product_id','parent_product_id']], on = 'product_id', how = 'left')

# For LIN AND AMT : 
ln3_2_35 = (
    ln3.filter(pl.col('parent_product_id').is_in([2,35]))
    .group_by(['IID','PlanID','parent_product_id'])
    .agg(
        pl.col('TUF_c').sum().alias('TUF_c'),pl.col('TUF_p').sum().alias('TUF_p'),
        pl.col('IRWD_FGN_NAME').first().alias('IRWD_FGN_NAME'),
        pl.col('plan_type').first().alias('plan_type'),
        pl.col('plan_class').first().alias('plan_class')
        #pl.col('plan_class').unique().str.concat(' / ').alias('plan_class')
    )
    .rename({'parent_product_id':'product_id'})
)

# For LAX MARKET : 
ln3_1 = (
    ln3
    .group_by(['IID','PlanID'])
    .agg(
        pl.col('TUF_c').sum().alias('TUF_c'),pl.col('TUF_p').sum().alias('TUF_p'),
        pl.col('IRWD_FGN_NAME').first().alias('IRWD_FGN_NAME'),
        pl.col('plan_type').first().alias('plan_type'),
    )
    .with_columns(pl.lit(1).alias('product_id').cast(pl.Int64),pl.lit('N_a').alias('plan_class'))
    .select(ln3_2_35.columns)
)
ln3_all_products = ln3.select(ln3_2_35.columns).vstack(ln3_2_35).vstack(ln3_1)

In [9]:
# Ranking 
ibsc_ranking = (
    ln3_1
    .with_columns(pl.col("TUF_c").rank("ordinal",descending=True).over(["IID"]).alias("rank"))
    .filter(pl.col('rank') <= 5)
    .sort(['IID','rank'],maintain_order=True)
    .select(['IID','PlanID','IRWD_FGN_NAME','plan_type']) 
    .with_columns(fl = 1)
    .join(prod_mapping.select('product_id').with_columns(fl = 1),on = 'fl',how = 'left')
    .drop('fl')
)


ln4 = (
    ibsc_ranking
    .join(
        ln3_all_products.select(['IID','PlanID','product_id','TUF_c','TUF_p']),
        on = ['IID','PlanID','product_id'], how = 'left'
    )
    .join(
        ln3_all_products.select(['PlanID','product_id','plan_class']).unique(),
        on = ['PlanID','product_id'], how = 'left'
    )
    .with_columns(pl.col('TUF_c').fill_null(pl.lit(0)),pl.col('TUF_p').fill_null(pl.lit(0)))
)

###### Formula based columns - 

In [10]:
# volume growth percentage - 
ln5 = (
    ln4
    .with_columns(
        ((pl.col('TUF_c') / pl.col('TUF_p')) - 1).alias('vol_growth_prc').round(10)
    )
)

In [11]:
# shr-
def get_shr(df):
    # Isolate Row with Product = 1

    df_1 = (
        df
        .filter(pl.col('product_id')==1)
        .select(['IID','PlanID','TUF_c'])
        .rename({'TUF_c':'lax_TUF_c'})
    )

    df = (
        df
        .join(df_1, on =['IID','PlanID'],how = 'left')
        .with_columns(
            (pl.col('TUF_c')/pl.col('lax_TUF_c')).round(10).alias('shr')
        )
        .drop('lax_TUF_c')
    )
    return (df)
    
ln5 = get_shr(ln5)

In [12]:
# vol growth ind - 
ln5 = (
    ln5
    .with_columns(
        pl.when(pl.col('vol_growth_prc') > 0.02).then(pl.lit('P'))
        .when(pl.col('vol_growth_prc') < -0.02).then(pl.lit('Q'))
        .otherwise(None).alias('vol_growth_ind')
    )
    .drop()
)

##### Format and Table fixes -

In [19]:
# Feed Creation -
ln6 = (
    ln5
    .drop(['TUF_p','PlanID'])
    .with_columns(
        pl.col('vol_growth_prc').replace([np.nan, np.inf, -np.inf], '\\N'),
        pl.when(pl.col('vol_growth_prc').is_in([np.nan, np.inf, -np.inf])).then(pl.lit('\\N')).otherwise(pl.col('vol_growth_ind')).alias('vol_growth_ind'),
        pl.col('shr').replace([np.nan, np.inf, -np.inf,0], '\\N'),
        pl.col('TUF_c').round(3).replace(0.0,'\\N'),
        pl.col('plan_type').str.to_uppercase()
    )
    .rename(
        {
            'IID' : 'DOCTORID',
            'IRWD_FGN_NAME' : 'PAYER',
            'plan_type' : 'PAYERTYPE',
            'plan_class' : 'COVERAGESTATUS',
            'TUF_c' : 'VOL',
            'product_id' : 'PRODUCT_ID',
            'vol_growth_prc' : 'VOL_GROWTH_PRC',
            'shr' : 'SHR',
            'vol_growth_ind' : 'VOL_GROWTH_IND'
        }
    )
    .select(['DOCTORID','PRODUCT_ID','PAYER','PAYERTYPE','COVERAGESTATUS','VOL','VOL_GROWTH_PRC','SHR','VOL_GROWTH_IND'])
)

###### Export -

In [15]:
#Exporting Feeds-
OUT = 's3://vortex-staging-a65ced90/BIT/output/Doc360/'
ln6.to_pandas().to_csv(f'{OUT}Doc360_ManagedCare_Feed.txt', sep='|',lineterminator='\r\n',index=False)
print('Doc360_ManagedCare_Feed.txt Exported !')

Doc360_ManagedCare_Feed.txt Exported !


---

# To Do :
    Investigate COVERAGESTATUS Diff - fixed logic but still mismatch
    VOL_GROWTH_IND - Check formula | Values most likely wrong
    check delta of 300k rows
    

In [16]:
#IID to check = 18113644

In [17]:
# s = 9030348
# p = 9316872
# delta = 286524 | 300k rows

In [21]:
ln6.head()

DOCTORID,PRODUCT_ID,PAYER,PAYERTYPE,COVERAGESTATUS,VOL,VOL_GROWTH_PRC,SHR,VOL_GROWTH_IND
i64,i64,str,str,str,str,str,str,str
232,1,"""Singlecare Hlth Savings Card …","""COMMERICAL""","""N_a""","""1.01""","""\N""","""1.0""","""\N"""
232,2,"""Singlecare Hlth Savings Card …","""COMMERICAL""","""N_a""","""\N""","""\N""","""\N""","""\N"""
232,4,"""Singlecare Hlth Savings Card …","""COMMERICAL""","""N_a""","""\N""","""\N""","""\N""","""\N"""
232,5,"""Singlecare Hlth Savings Card …","""COMMERICAL""","""N_a""","""\N""","""\N""","""\N""","""\N"""
232,3,"""Singlecare Hlth Savings Card …","""COMMERICAL""","""N_a""","""\N""","""\N""","""\N""","""\N"""
